In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')


In [ ]:
df_train = pd.read_csv('bigmart_train.csv')

In [ ]:
df_train.shape

In [ ]:
df_train.head(3)

In [ ]:
df_train.isna().sum()

In [ ]:
df_train['Item_Fat_Content'].unique()

In [ ]:
sns.countplot(x='Item_Fat_Content',data=df_train)

In [ ]:
df_train['Item_Type'].unique()

In [ ]:
sns.countplot(x='Item_Type',data=df_train)

In [ ]:
df_train['Outlet_Size'].unique()

In [ ]:
df_train['Outlet_Size'].value_counts()

In [ ]:
sns.countplot(x='Outlet_Size',data=df_train)

In [ ]:
df_train['Outlet_Establishment_Year'].value_counts()

In [ ]:
sns.countplot(x='Outlet_Establishment_Year',data=df_train)

In [ ]:
df_train['Outlet_Location_Type'].value_counts()

In [ ]:
sns.countplot(x='Outlet_Location_Type',data=df_train)

In [ ]:
df_train['Outlet_Type'].value_counts()

In [ ]:
sns.countplot(x='Outlet_Type',data=df_train)

In [ ]:
df_train.describe()

In [ ]:
df_train['Outlet_Age']=2021-df_train['Outlet_Establishment_Year']

### Missing Values:

In [ ]:
df_train.isna().sum()

In [ ]:
df_train['Item_Weight']= df_train['Item_Weight'].fillna(df_train['Item_Weight'].mean())

In [ ]:
# df_train['Outlet_Size'].mode()[0]
df_train['Outlet_Size'].value_counts()

In [ ]:
df_train['Outlet_Size'].fillna('Medium',inplace=True)

In [ ]:
df_train.isna().sum()

### Outilers:

In [ ]:
a,b = plt.subplots(1,3)
df_train['Item_Weight'].plot(kind='box',ax=b[0])
df_train['Item_Visibility'].plot(kind='box',ax=b[1])
df_train['Item_MRP'].plot(kind='box',ax=b[2])

In [ ]:

df_train['Item_Visibility'].hist()

<img src=IQR_Outliers.png>

In [ ]:
Q1 = df_train['Item_Visibility'].quantile(.25)

Q3 = df_train['Item_Visibility'].quantile(.75)

IQR = Q3 -Q1

df2= df_train.query('(@Q1- 1.5* @IQR) <= Item_Visibility <= (@Q3 +1.5* @IQR)')

In [ ]:
df2.shape

In [ ]:
Q1-1.5*IQR, Q1, IQR, Q3,   Q3+1.5*IQR

In [ ]:
df2['Item_Visibility'].describe()

In [ ]:
df_train=df2

In [ ]:
df_train.shape

In [ ]:
df_train['Item_Fat_Content'].unique()

In [ ]:
df_train['Item_Fat_Content']= df_train['Item_Fat_Content'].replace(['low fat','LF'],'Low Fat')

In [ ]:
df_train['Item_Fat_Content'].unique()

In [ ]:
df_train['Item_Fat_Content']= df_train['Item_Fat_Content'].replace('reg','Regular')

In [ ]:
df_train['Item_Fat_Content'].unique()

In [ ]:
sns.countplot(x='Item_Fat_Content',data=df_train)

In [ ]:
df_train['Item_Visibility'].value_counts()

In [ ]:

df_train['Item_Visibility'].hist()

### converting into Categorical values:

In [ ]:
df_train['Item_Visibility']=pd.cut(df_train['Item_Visibility'],[-1,0.075,0.15,0.2],labels=['Low Viz','Viz', 'High Viz'])


In [ ]:
df_train['Item_Visibility'].value_counts()

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train['Item_Identifier']=df_train['Item_Identifier'].apply(lambda x:x[0:2])

In [ ]:
df_train.head()

In [ ]:
df_train['Item_combined']=df_train['Item_Identifier'].map({'FD':'Food','DR':'Drink','NC':'Non edible'})

In [ ]:
df_train

In [ ]:
df_train['Item_Fat_Content'].value_counts()

In [ ]:
for i in range(len(df_train['Item_combined'])):
    s=df_train['Item_combined'].iloc[i]
    if s=="Non edible":
        df_train['Item_Fat_Content'].iloc[i]="Non consumable" #non edible

In [ ]:
df_train['Item_Fat_Content'].value_counts()

In [ ]:
df_train.head()

In [ ]:
var=['Item_Fat_Content','Item_Visibility','Outlet_Size' ,'Outlet_Location_Type','Outlet_Age','Item_Type']

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()

In [ ]:
for i in var:
    df_train[i]=le.fit_transform(df_train[i])

In [ ]:
df_train

### One Hot Encoding:

In [ ]:
df_train=pd.get_dummies(df_train,columns=['Outlet_Type','Item_combined'])

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
df_train.head()

***
### X, y:

In [ ]:
X=df_train.drop(['Item_Identifier','Outlet_Identifier','Outlet_Establishment_Year','Item_Outlet_Sales'],axis=1)


In [ ]:
X

In [ ]:
y=df_train['Item_Outlet_Sales']

In [ ]:
y

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 0)

In [ ]:
X.shape , X_train.shape, y_train.shape, X_test.shape, y_test.shape

### multi linear regression

In [ ]:
from sklearn.linear_model import LinearRegression
lr= LinearRegression()
lr.fit(X_train, y_train)
y_pred= lr.predict(X_test)
lr.score(X_test,y_test)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

sqrt(mean_squared_error(y_test,y_pred))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 2000, random_state = 0)
regressor.fit(X_train, y_train)

In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
from sklearn import metrics
import numpy as np

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(max_depth=5)
regressor.fit(X_train, y_train)

In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)

In [ ]:
y_pred = regressor.predict(X_test)


In [ ]:
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_auc_score
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [ ]:
sfs = SFS(RandomForestRegressor(n_estimators=1000, random_state=0, n_jobs = -1),
         k_features = 15,
          forward= True,
          floating = False,
          verbose= 2,
          scoring= 'neg_root_mean_squared_error',
          cv = 4,
          n_jobs= -1
         ).fit(X_train, y_train)

In [ ]:
sfs.k_feature_names_

In [ ]:
sfs.k_feature_idx_

In [ ]:
sfs.k_score_ 

In [ ]:
pd.DataFrame.from_dict(sfs.get_metric_dict()).T

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly_reg = PolynomialFeatures(degree=2)
X_poly = poly_reg.fit_transform(X_train)
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_poly, y_train)

In [ ]:
X_test_poly= poly_reg.fit_transform(X_test)
y_pred= lin_reg_2.predict(X_test_poly)
lin_reg_2.score(X_test_poly,y_test)

In [ ]:
sqrt(mean_squared_error(y_test,y_pred))

## drop unnecessary

In [ ]:
X1=X.drop([ 'Outlet_Type_Supermarket Type1','Item_combined_Drink','Item_combined_Non edible'],axis=1)


In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y, test_size = .3, random_state = 0)

## HIGHEST ACCURACY MODEL

In [ ]:
poly_reg = PolynomialFeatures(degree=2)
X_poly1 = poly_reg.fit_transform(X_train1)
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_poly1, y_train1)

In [ ]:
X_test_poly1= poly_reg.fit_transform(X_test1)
y_pred= lin_reg_2.predict(X_test_poly1)
lin_reg_2.score(X_test_poly1,y_test1)

In [ ]:
sqrt(mean_squared_error(y_test1,y_pred))

## TRIED SOMETHING MORE

*** 
### select from model best features

In [ ]:
from sklearn.feature_selection import SelectFromModel

In [ ]:
from sklearn.feature_selection import RFE
sel = RFE(RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=-1), n_features_to_select = 12)
sel.fit(X_train, y_train)

In [ ]:
sel.get_support()

In [ ]:
X_train_rfe = sel.transform(X_train)
X_test_rfe = sel.transform(X_test)

### rfr

In [ ]:
regressor = RandomForestRegressor(n_estimators = 2000, random_state = 0)
regressor.fit(X_train_rfe, y_train)

In [ ]:
y_pred = regressor.predict(X_test_rfe)
from sklearn import metrics
import numpy as np

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

## poly

In [ ]:
poly_reg = PolynomialFeatures(degree=2)
X_poly1 = poly_reg.fit_transform(X_train_rfe)
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_poly1, y_train)

In [ ]:
X_test_poly1= poly_reg.fit_transform(X_test_rfe)
y_pred= lin_reg_2.predict(X_test_poly1)
lin_reg_2.score(X_test_poly1,y_test)

## gradient boost

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
model= GradientBoostingRegressor()
model.fit(X_train1, y_train1)
model.score(X_test1,y_test1)

In [ ]:
import xgboost as xgb
model=xgb.XGBRegressor()
model.fit(X_train1, y_train1)
model.score(X_test1,y_test1)

## LGB

In [ ]:
import lightgbm as lgb
train_data=lgb.Dataset(X_train1,label=y_train1)
params = {'learning_rate':0.001}
model= lgb.train(params, train_data, 100)
from sklearn.metrics import mean_squared_error
y_pred=model.predict(X_test1)
rmse=mean_squared_error(y_pred,y_test1)**0.5
print(rmse)

### ENSEMBLE

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from matplotlib import pyplot
from sklearn.ensemble import StackingRegressor
def get_models():
    models = dict()
    models['knn'] = KNeighborsRegressor()
    models['cart'] = DecisionTreeRegressor()
    models['svm'] = SVR()
    models['stacking'] = get_stacking()
    return models
def get_stacking():

    level0 = list()
    level0.append(('knn', KNeighborsRegressor()))
    level0.append(('cart', DecisionTreeRegressor()))
    level0.append(('svm', SVR()))
    # define meta learner model
    level1 = LinearRegression()

    model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
    return model
 
# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
    return scores
 

# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, X1, y)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

### knn

In [ ]:
from sklearn import neighbors
from sklearn.metrics import mean_squared_error 
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline
rmse_val = [] #to store rmse values for different k
for K in range(20):
    K = K+1
    model = neighbors.KNeighborsRegressor(n_neighbors = K)

    model.fit(X_train1, y_train1)  #fit the model
    pred=model.predict(X_test1) #make prediction on test set
    error = sqrt(mean_squared_error(y_test1,pred)) #calculate rmse
    rmse_val.append(error) #store rmse values
    print('RMSE value for k= ' , K , 'is:', error)

In [ ]:
curve = pd.DataFrame(rmse_val) #elbow curve 
curve.plot()

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'n_neighbors':[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]}

knn = neighbors.KNeighborsRegressor()

model = GridSearchCV(knn, params, cv=5)
model.fit(X_train1,y_train1)
model.best_params_

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
import numpy as np
rng = np.random.RandomState(1)
regr_1 = DecisionTreeRegressor(max_depth=5)

regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                          n_estimators=300, random_state=rng)

In [ ]:
regr_1.fit(X_train1, y_train1)
regr_2.fit(X_train1, y_train1)

In [ ]:
y_1 = regr_1.predict(X_test1)
y_2 = regr_2.predict(X_test1)


In [ ]:
print('Root Mean Squared Error: DT', np.sqrt(metrics.mean_squared_error(y_test1, y_1)))
print('Root Mean Squared Error: ADABOOST DT', np.sqrt(metrics.mean_squared_error(y_test1, y_2)))